# Murder Mystery

"*As the clock strikes midnight in the Old Tudor Mansion, a raging storm rattles the shutters and fills the house with the sound of thunder...*" ([MBMLBook](http://www.mbmlbook.com/MurderMystery.html))

This has been updated in Oct 2018 (post Infer.NET open source launch!) to use: 
* the Paket dependency manager to get the package(s) from nuget
* Paket to get the `FSharpWrapper` from github
* the revised namespace `Microsoft.ML.Probabilistic` (was `MicrosoftResearch.Infer`)

## Links

http://infernet.azurewebsites.net/default.aspx    
http://www.mbmlbook.com/MurderMystery.html    
https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/fun-probabilistic_programming_obt_january_2013.pdf     
https://github.com/prgoodwin/HabilisX/blob/master/Infer.NET%202.5/Samples/Fun/MurderMystery/MurderMystery.fs  (Uses Infer.NET Fun.)    
http://mlss.tuebingen.mpg.de/2013/2013/bishop_slides.pdf   

## Preamble

### Install packages

https://fsprojects.github.io/Paket/

Packages in F# can be installed via `nuget.org` using the Paket dependency manager.
To install packages from NuGet you should first load the Packet package manager:

In [1]:
#load "Paket.fsx"

You can then have Paket install the packages:

In [2]:
Paket.Package
  [   "Microsoft.ML.Probabilistic"
      "Microsoft.ML.Probabilistic.Compiler"
      //"Microsoft.ML.Probabilistic.FSharp"  // Not on nuget, yet?
      "NETStandard.Library" 
  ]

In [3]:
#load "Paket.Generated.Refs.fsx"  // Do we need this?

### `FSharpWrapper`

See  https://github.com/dotnet/infer/blob/master/src/FSharpWrapper/FSharpWrapper.fs   
https://github.com/fsprojects/IfSharp/issues/146 (Thanks for help, Colin Gravill!)   
https://fsprojects.github.io/Paket/github-dependencies.html

In [5]:
Paket.GitHub ["dotnet/infer src/FSharpWrapper/FSharpWrapper.fs"]

In [6]:
//#load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs"

### Check `paket` files

Project dependencies file:

In [6]:
open System
open System.IO

In [ ]:
File.ReadAllText(@"../../IfSharp/bin/paket.dependencies")

The `paket install` command will analyze your dependencies and automatically generate a `paket.lock` file:

In [ ]:
File.ReadAllText(@"../../IfSharp/bin/paket.lock")

### Assembly search paths and references

In [7]:
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic.Compiler/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/NETStandard.Library/build/netstandard2.0/ref"

In [8]:
#r "Microsoft.ML.Probabilistic"
#r "Microsoft.ML.Probabilistic.Compiler"
#r "netstandard"

### Import declarations for modules or namespaces 

In [9]:
open Microsoft.ML.Probabilistic
open Microsoft.ML.Probabilistic.Algorithms
open Microsoft.ML.Probabilistic.Distributions // VectorGaussian, Gamma
open Microsoft.ML.Probabilistic.Factors
open Microsoft.ML.Probabilistic.Math          // Vector
open Microsoft.ML.Probabilistic.Models        // Variable, VariableArray, Range

In [10]:
#load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs"

In [11]:
open Microsoft.ML.Probabilistic.FSharp

### Murder Mystery

* Either Alice or Bob did it (done it = "dunnit")
* Alice dunnit 30%, Bob dunnit 70%
* Alice uses gun 3%, uses pipe 97%
* Bob uses gun 80%, uses pipe 20%

In [12]:
let aliceDunnit:Variable<bool> = Variable.Bernoulli(0.30).Named("aliceDunnit")

[Imperative Statement Blocks in F#](https://dotnet.github.io/infer/userguide/Imperative%20Statement%20Blocks%20in%20FSharp.html)     
[Branching on variables to create mixture models](https://dotnet.github.io/infer/userguide/Branching%20on%20variables%20to%20create%20mixture%20models.html)  

We learn from the [user guide](https://dotnet.github.io/infer/userguide/Imperative%20Statement%20Blocks%20in%20FSharp.html) that the `Variable.IfBlock` method returns an `IfBlock` for creating if then else statements which are dependant on the value of a `ariable<bool>`. This is a function with type signature:   
**`Variable.IfBlock: Variable<bool>->(Variable<bool>->unit)->Variable<bool>->unit)->unit`**   
where, the first argument is a boolean variable, the second argument is a function to be applied if the Boolean Variable has value true and the third argument is a function to be applied if the Boolean Variable has value false.

In [ ]:
// If Block function  
let f1 (vb1: Variable<bool>) =  Variable.Bernoulli(0.03)
// IfNot Block function  
let f2 (vb2: Variable<bool>) =  Variable.Bernoulli(0.80)

In [81]:
let mystery () =
    let aliceDunnit = Variable.Bernoulli(0.30).Named("aliceDunnit")
    let withGun = Variable.IfBlock aliceDunnit Variable.Bernoulli(0.03) Variable.Bernoulli(0.80)
    aliceDunnit, withGun

Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
This expression was expected to have type
    'Variable<bool> -> unit'    
but here has type
    'Variable<bool>'    
This expression was expected to have type
    'Variable<bool> -> unit'    
but here has type
    'Variable<bool>'    

In [32]:
let mystery () =
    let aliceDunnit = Variable.Bernoulli(0.30)
    let withGun =
        if aliceDunnit
        then Variable.Bernoulli(0.03)
        else Variable.Bernoulli(0.80)
    aliceDunnit, withGun

This expression was expected to have type
    'bool'    
but here has type
    'Variable<bool>'    

In [31]:
// Pipe at scene - now Alice dunnit 69%
let PipeFoundAtScene () =
    let aliceDunnit, withGun = mystery ()
    observe(withGun = false)
    aliceDunnit, withGun

The value or constructor 'mystery' is not defined.
The value or constructor 'observe' is not defined. Maybe you want one of the following:
   IObserver
   Observable
   ObservableModule
   IObservable

### Clinical trial

In [72]:
// Data from a clinical trial  
let controlGroup = Variable.Observed<bool>([|false; false; true; false; false|])  
let treatedGroup = Variable.Observed<bool>([|true; false; true; true; true |])  
let i = controlGroup.Range  
let j = treatedGroup.Range  

In [73]:
// Prior on being an effective treatment  
let isEffective = Variable.Bernoulli(0.5).Named("isEffective");  
let probIfTreated = ref (Variable.New<float>())  
let probIfControl = ref (Variable.New<float>())

In [74]:
// If Block function  
let f1 (vb1: Variable<bool>) =  
    probIfControl := Variable.Beta(1.0, 1.0).Named("probIfControl")  
    let controlGroup = Variable.AssignVariableArray controlGroup i (fun i -> Variable.Bernoulli(!probIfControl)) 
    let treatedGroup = Variable.AssignVariableArray treatedGroup j (fun j -> Variable.Bernoulli(!probIfTreated))  
    ()  

In [75]:
// IfNot Block function  
let f2 (vb2: Variable<bool>) =  
    let probAll = Variable.Beta(1.0, 1.0).Named("probAll")  
    let controlGroup = Variable.AssignVariableArray controlGroup i (fun i -> Variable.Bernoulli(probAll))  
    let treatedGroup = Variable.AssignVariableArray treatedGroup j (fun j -> Variable.Bernoulli(probAll))  
    ()  

In [76]:
// Call IfBlock  
let test = Variable.IfBlock isEffective f1 f2

In [77]:
test